In [32]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [73]:
telecom_data = pd.read_csv(r"C:\Users\kidus\synergyTelecom\data\WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [98]:
no_tenure = telecom_data[telecom_data.TotalCharges == " "]

cleaned_telecom_data = telecom_data[~telecom_data.index.isin(no_tenure.index)].copy()
cleaned_telecom_data["TotalCharges"] = cleaned_telecom_data["TotalCharges"].astype(float)

cleaned_telecom_data.MonthlyCharges.dtype

dtype('float64')

In [ ]:
sns.histplot(data = cleaned_telecom_data, x = "TotalCharges", kde= True)